In [1]:
!pip install telebot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 15.4 MB/s eta 0:00:00


In [ ]:
import cv2
import telebot
import numpy as np
import tensorflow as tf
from telebot import types


mybot = telebot.TeleBot("token", parse_mode=None)

@mybot.message_handler(commands=['start'])
def send_welcome(message):
    msg = mybot.send_message(message.chat.id,"Hi "+str(message.chat.first_name)+" \n" + "Welcome to 'fellow and clergyman Recognition Bot'"+" \n"+
                            "This bot can recognize fellow and clergyman."+'\n'+"/fellow_and_clergyman_Recognition")


@mybot.message_handler(commands=['fellow_and_clergyman_Recognition'])
def send_photo(message):
    msg = mybot.reply_to(message,"Please send me a photo of a person.")
    mybot.register_next_step_handler(msg,recognize_flowers)

def recognize_flowers(message):
    my_model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/PyLearn7-Assignment54/fellow and clergyman/weights/fellow_and_clergyman.keras')

    fileID = message.photo[-1].file_id
    file_info = mybot.get_file(fileID)
    downloaded_file = mybot.download_file(file_info.file_path)

    with open("image.jpg", 'wb') as new_file:
        new_file.write(downloaded_file)

    image = cv2.imread("image.jpg")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image , (224, 224))
    image = image / 255.0
    image = image.reshape(1, 224, 224, 3)

    output = my_model.predict(image)
    predicted_class = np.argmax(output)

    labels = ['clergyman', 'fellow']

    label = labels[predicted_class]
    print(label)

    if label == 'clergyman':
        mybot.send_message(message.chat.id, "He is a clergyman.")
    else:
        mybot.send_message(message.chat.id, "He/She is our fellow.")

mybot.enable_save_next_step_handlers(delay=2)
mybot.load_next_step_handlers()
mybot.infinity_polling()

1/1 ━━━━━━━━━━━━━━━━━━━━ 19s 19s/step
clergyman
